In [1]:
import re
from urllib.request import Request, urlopen
from urllib.parse import urljoin
import pandas as pd
from bs4 import BeautifulSoup as BS
from tqdm import tqdm

In [2]:
# JIS color data
# source: https://www.i-iro.com/dic/tag/jis

srcurl = "https://www.i-iro.com/dic/tag/jis"
x = urlopen(Request(srcurl)).read()
soup = BS(x, "html.parser")
links = [urljoin(srcurl, a.get("href")) for a in soup.select("ul.archive-list li a")]

def clean_color(url):
  soup = BS(urlopen(Request(url)).read(), "html.parser")
  # find category this color belong to
  categories = [a.text.strip() for a in soup.select("div.aioseo-breadcrumbs a")]
  categories = [a for a in categories if a != "色彩図鑑"]
  assert len(categories) == 2, url

  # find data of this color
  headers = soup.select("table.color-data th")
  data = {}
  for h in headers:
    value = h.find_next("td")
    assert value is not None, url
    header = h.text.strip()
    value = value.text.strip()
    # to English
    header_en = (
      "colorname" if header == "色の名前" else
      "reading" if header.startswith("読み") else
      "description" if header.startswith("由来") else
      "feature" if header == "系統色名" else
      "munsell" if header == "マンセル値" else
      "rgb" if header == "webcolor" else
      "rgb_num" if header == "RGB" else
      "cmyk_num" if header == "CMYK" else
      None
    )
    assert header_en is not None, url    
    data[header_en] = value
  data["rgb"] = data["rgb"].upper()
  del data["rgb_num"]
  del data["cmyk_num"]
    
  data["category_country"] = categories[0][:2].strip()
  data["category_color"] = re.sub(r"系.*", "", categories[1]).strip()
  
  tmp = re.search(r"[\(（]([^)\)]+)[\)）]", data["feature"])
  assert tmp is not None, url
  data["feature_word"] = tmp.group(1).strip()
  data["feature_code"] = data["feature"][:tmp.start()].strip()
  return data

In [3]:
clean_color(links[0])

{'colorname': '青',
 'reading': 'あお',
 'description': '光の三原色のひとつ',
 'feature': 'vv-B (鮮やかな青)',
 'munsell': '10B 4/14',
 'rgb': '#0095D9',
 'category_country': '日本',
 'category_color': '青',
 'feature_word': '鮮やかな青',
 'feature_code': 'vv-B'}

In [4]:
data = [clean_color(u) for u in tqdm(links)]
data = pd.DataFrame(data)
data

100%|█████████████████████████████████████████| 264/264 [01:00<00:00,  4.37it/s]


,colorname,reading,description,feature,munsell,rgb,category_country,category_color,feature_word,feature_code
0,青,あお,光の三原色のひとつ,vv-B (鮮やかな青),10B 4/14,#0095D9,日本,青,鮮やかな青,vv-B
1,空色,そらいろ,晴れた空のような明るい青,lt-B (明るい青),9B 7.5/5.5,#A0D8EF,日本,青,明るい青,lt-B
2,水色,みずいろ,澄んだ水のような薄い青,pl-gB (薄い緑みの青),6B 8/4,#BCE2E8,日本,青,薄い緑みの青,pl-gB
3,瑠璃色,るりいろ,宝石の瑠璃（ラピスラズリ）の色,dp-pB (濃い紫みの青),6PB 3/8,#1D50A2,日本,青,濃い紫みの青,dp-pB
4,瑠璃紺,るりこん,宝石の瑠璃を由来とする色名,dp-pB (濃い紫みの青),6PB 3/8,#18448E,日本,青,濃い紫みの青,dp-pB
...,...,...,...,...,...,...,...,...,...,...
259,シルバーグレー,silver grey,銀のような光沢を感じる灰色,ltGy (明るい灰色),N 6.5,#AFAFB0,世界,モノクロ,明るい灰色,ltGy
260,スレートグレー,slate grey,スレート（粘板岩）の色,dkGy (暗い灰色),2.5PB 3.5/0.5,#636062,世界,モノクロ,暗い灰色,dkGy
261,チャコールグレー,charcoal grey,木炭や炭の色,p-dkGy (紫みの暗い灰色),5P 3/1,#4E4449,世界,モノクロ,紫みの暗い灰色,p-dkGy
262,スチールグレー,steel grey,鉄を主成分とする合金、スチール（鋼）の色,p-mdGy (紫みの灰色),5P 4.5/1,#736D71,世界,モノクロ,紫みの灰色,p-mdGy


In [5]:
data.category_color.value_counts().sort_index()

イエロー    17
グリーン    17
パープル     9
ブラウン    19
ブルー     19
モノクロ    14
レッド     27
白黒      15
紫       14
緑       22
茶       29
赤       18
青       25
黄       19
Name: category_color, dtype: int64

In [6]:
# translate color category
tmp = data.category_color.copy()
tmp[data.category_color.isin(("イエロー", "黄"))] = "yellow"
tmp[data.category_color.isin(("グリーン", "緑"))] = "green"
tmp[data.category_color.isin(("パープル", "紫"))] = "purple"
tmp[data.category_color.isin(("ブラウン", "茶"))] = "brown"
tmp[data.category_color.isin(("ブルー", "青"))] = "blue"
tmp[data.category_color.isin(("モノクロ", "白黒"))] = "monochrome"
tmp[data.category_color.isin(("レッド", "赤"))] = "red"
display(pd.crosstab(data.category_color, tmp))
data["category_color_en"] = tmp

category_color,blue,brown,green,monochrome,purple,red,yellow
category_color,,,,,,,
イエロー,0,0,0,0,0,0,17
グリーン,0,0,17,0,0,0,0
パープル,0,0,0,0,9,0,0
ブラウン,0,19,0,0,0,0,0
ブルー,19,0,0,0,0,0,0
モノクロ,0,0,0,14,0,0,0
レッド,0,0,0,0,0,27,0
白黒,0,0,0,15,0,0,0
紫,0,0,0,0,14,0,0


In [7]:
data[data.category_color.isin(["白黒", "モノクロ"])]

,colorname,reading,description,feature,munsell,rgb,category_country,category_color,feature_word,feature_code,category_color_en
235,生成色,きなりいろ,糸や生地のそのままの色,r-y-Wt (赤みの黄みの白),10YR 9/1,#FBFAF3,日本,白黒,赤みの黄みの白,r-y-Wt,monochrome
236,象牙色,ぞうげいろ,象の牙の色,y-plGy (黄みの薄い灰色),2.5Y 8.5/1.5,#F8F5E3,日本,白黒,黄みの薄い灰色,y-plGy,monochrome
237,胡粉色,ごふんいろ,貝殻を粉にした胡粉の色,y-Wt (黄みの白),9.5N,#FFFFFC,日本,白黒,黄みの白,y-Wt,monochrome
238,白,しろ,明確な状態を表す「しろし」に由来,Wt (白),N9.5,#FFFFFE,日本,白黒,白,Wt,monochrome
239,黒,くろ,「暗い」や「暮れる」が語源,Bk (黒),N1.5,#130012,日本,白黒,黒,Bk,monochrome
240,砂色,すないろ,砂のような色,lg-Y (明るい灰みの黃),2Y 7.5/2,#DCD3B2,日本,白黒,明るい灰みの黃,lg-Y,monochrome
241,灰色,はいいろ,灰のような色,mdGy (灰色),N5,#767676,日本,白黒,灰色,mdGy,monochrome
242,鉛色,なまりいろ,金属の鉛のような色,b-mdGy (青みの灰色),2.5PB 5/1,#7A7C7D,日本,白黒,青みの灰色,b-mdGy,monochrome
243,鼠色,ねずみいろ,元は鼠の毛の色。灰色全般を表すようになった,mdGy (灰色),N5.5,#949495,日本,白黒,灰色,mdGy,monochrome
244,茶鼠,ちゃねず,茶色みの鼠色,y-r-mdGy (黄みの赤みの灰色),5YR 6/1,#A99E93,日本,白黒,黄みの赤みの灰色,y-r-mdGy,monochrome


In [9]:
data.to_csv("../jis.csv", encoding="utf8", index=False)